# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [157]:
# 1. Read `tic-tac-toe.csv` into a dataframe.

import pandas as pd
import numpy as np

game = pd.read_csv('tic-tac-toe.csv')
game.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [158]:
# 2. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
game.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object

In [159]:
# 3. Convert the categorical values to numeric in all columns.
game2 = game.copy()

game2.replace({'x': 1, 'o': 2, 'b': 0}, inplace=True)
game2.astype("float32")

C:\Users\crvid\AppData\Local\Temp\ipykernel_26140\3521100472.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  game2.replace({'x': 1, 'o': 2, 'b': 0}, inplace=True)


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0
1,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0
2,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0
3,1.0,1.0,1.0,1.0,2.0,2.0,2.0,0.0,0.0,1.0
4,1.0,1.0,1.0,1.0,2.0,2.0,0.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
953,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,0.0
954,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0
955,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,0.0
956,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,0.0


In [160]:
# 4. Separate the inputs and outputs.
X = game2.drop(columns=["class"]) # inputs
y = game2["class"] # outputs

In [161]:
# 5. Normalize the input data.
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() # Initialize the scaler

X_normalized = scaler.fit_transform(X) # Fit and transform 

X_normalized


array([[0.5, 0.5, 0.5, ..., 0.5, 1. , 1. ],
       [0.5, 0.5, 0.5, ..., 1. , 0.5, 1. ],
       [0.5, 0.5, 0.5, ..., 1. , 1. , 0.5],
       ...,
       [1. , 0.5, 1. , ..., 0.5, 1. , 0.5],
       [1. , 0.5, 1. , ..., 0.5, 1. , 0.5],
       [1. , 1. , 0.5, ..., 1. , 0.5, 0.5]])

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [162]:
# 1. Split the training and test data.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [163]:
# 2. Create a `Sequential` model.
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential() # Initialize the model

In [164]:
# 3. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. 
# Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
""" 9 is the number of columns in X"""
model.add(Dense(80, activation='relu', input_shape=(9,))) 
model.add(Dense(30, activation='relu'))
model.add(Dense(50, activation='relu'))

model.add(Dense(2, activation='softmax'))

c:\Users\crvid\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [165]:
# 4. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [166]:
# 5. Fit the training data.
history = model.fit(X_train, y_train, batch_size=9, epochs=30, verbose=1, validation_data=(X_test, y_test))

Epoch 1/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5262 - loss: 0.7170 - val_accuracy: 0.6510 - val_loss: 0.6335
Epoch 2/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6470 - loss: 0.6288 - val_accuracy: 0.6562 - val_loss: 0.5993
Epoch 3/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6595 - loss: 0.5969 - val_accuracy: 0.6562 - val_loss: 0.5735
Epoch 4/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6931 - loss: 0.5688 - val_accuracy: 0.6615 - val_loss: 0.5309
Epoch 5/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7018 - loss: 0.5207 - val_accuracy: 0.6562 - val_loss: 0.4961
Epoch 6/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7380 - loss: 0.4944 - val_accuracy: 0.7396 - val_loss: 0.4665
Epoch 7/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7615 - loss: 0.4626 - val_accuracy: 0.7917 - val_loss: 0.4540
Epoch 8/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8087 - loss: 0.4154 - val_accuracy: 0.8021 - val_loss:

In [167]:
# 5. Evaluate your neural network model with the test data.
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8676 - loss: 0.2843
Test Loss: 0.25319504737854004
Test Accuracy: 0.8802083134651184


In [168]:
# 6. Save your model as `tic-tac-toe.model`.
model.save('tic-tac-toe.keras')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [169]:
# Load your saved model
from tensorflow.keras.models import load_model

saved_model = load_model('tic-tac-toe.keras')

In [170]:
# Make predictions on a few random rows in the test dataset
# Select a few random rows from the test dataset (e.g., 5 random rows)
random_indices = np.random.choice(X_test.shape[0], size=5, replace=False)
X_sample = X_test[random_indices]
y_sample = y_test.iloc[random_indices]

# Make predictions
predictions = saved_model.predict(X_sample)

# Convert predictions to class labels (argmax because of softmax)
predicted_labels = np.argmax(predictions, axis=1)

# Print actual and predicted values
print("Actual Labels: ", y_sample.values)
print("Predicted Labels: ", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Actual Labels:  [False  True False  True  True]
Predicted Labels:  [1 1 0 1 0]


In [171]:
# Check if the predictions are correct.
loss, accuracy = saved_model.evaluate(X_sample, y_sample)
print('loss', loss)
print('accuracy', accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step - accuracy: 0.6000 - loss: 0.4731
loss 0.4731330871582031
accuracy 0.6000000238418579


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [172]:
improved_model = Sequential()

# Adding more layers
improved_model.add(Dense(128, activation='relu', input_shape=(9,)))
improved_model.add(Dropout(0.2))
improved_model.add(Dense(64, activation='relu'))
improved_model.add(Dropout(0.2))
improved_model.add(Dense(32, activation='relu'))
improved_model.add(Dropout(0.2))
improved_model.add(Dense(16, activation='relu'))
improved_model.add(Dropout(0.2))

improved_model.add(Dense(2, activation='softmax'))


improved_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history = improved_model.fit(X_train, y_train, batch_size=9, epochs=100, verbose=1, validation_data=(X_test, y_test))

test_loss, test_accuracy = improved_model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.5816 - loss: 0.6826 - val_accuracy: 0.6510 - val_loss: 0.6383
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6604 - loss: 0.6442 - val_accuracy: 0.6510 - val_loss: 0.6235
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6208 - loss: 0.6480 - val_accuracy: 0.6510 - val_loss: 0.5973
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6637 - loss: 0.6037 - val_accuracy: 0.6510 - val_loss: 0.5741
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6600 - loss: 0.6154 - val_accuracy: 0.6771 - val_loss: 0.5583
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6816 - loss: 0.5612 - val_accuracy: 0.6562 - val_loss: 0.5072
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6800 - loss: 0.5473 - val_accuracy: 0.7708 - val_loss: 0.4999
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7580 - loss: 0.5240 - val_accuracy: 0.7604 - v

**Which approach(es) did you find helpful to improve your model performance?**

In [173]:
# Neural network improves the performance